In [1]:
import pandas as pd
from Bio import Entrez, SeqIO
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

Entrez.email = "l.singh@intbio.org"

In [2]:
with open("db_curated_server_info.txt", "r") as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()
    if line and not line.startswith("#"):
        key, value = line.split("=", 1)
        config[key] = value.strip()

server_name = config.get("server_name")
srever_port = int(config.get("srever_port"))
ssh_password = config.get("ssh_password")
ssh_username = config.get("ssh_username")
db_adress = config.get("db_adress")
db_port = int(config.get("db_port"))

In [3]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

34227


In [4]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [5]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [106]:
# add_histone = (
#     "INSERT INTO histone "
#     "(id, level, taxonomic_span, taxonomic_span_id, description, parent) "
#     "VALUES (%(id)s, %(level)s, %(taxonomic_span)s, %(taxonomic_span_id)s, %(description)s, %(parent)s)"
# )
# add_histone_description = (
#     "INSERT INTO histone_description "
#     "(summary, taxonomy, genes, evolution, expression, knock_out, function, sequence, localization, deposition, structure, interactions, disease, caveats) "
#     "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
# )
# add_publication = (
#     "INSERT INTO publication "
#     "(id, title, doi, author, year) "
#     "VALUES (%(id)s, %(title)s, %(doi)s, %(author)s, %(year)s)"
# )
# add_sequence_has_publication = (
#     "INSERT INTO sequence_has_publication "
#     "(sequence_accession, publication_id) "
#     "VALUES (%s, %s)"
# )
# add_alternate_names = (
#     "INSERT INTO alternative_name "
#     "(name, taxonomy, gene, splice, histone) "
#     "VALUES (%(name)s, %(taxonomy)s, %(gene)s, %(splice)s, %(histone)s)"
# )
# add_histone_has_publication = (
#     "INSERT INTO histone_has_publication "
#     "(histone_id, publication_id) "
#     "VALUES (%s, %s)"
# )

# Change accessions for genes H2BC8 and H2BC10

NP_003516 (H2BC10, 8346) и NP_003509 (H2BC8, 8339)

In [7]:
query = (
    "SELECT * FROM sequence s LEFT JOIN sequence_has_publication sp "
    "ON s.accession = sp.sequence_accession "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["accession"].str.contains("NP_003516")]

,accession,variant,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,sequence,variant_under_consideration,sequence_accession,publication_id
290,NP_003516.1,cH2B.3_(Homo_sapiens),None,8339,H2BC8,9606.0,Homo sapiens,Chordata,Mammalia,Mammalia,None,MPEPAKSAPAPKKGSKKAVTKAQKKDGKKRKRSRKESYSVYVYKVL...,None,NP_003516.1,12408966
291,NP_003516.1,cH2B.3_(Homo_sapiens),None,8339,H2BC8,9606.0,Homo sapiens,Chordata,Mammalia,Mammalia,None,MPEPAKSAPAPKKGSKKAVTKAQKKDGKKRKRSRKESYSVYVYKVL...,None,NP_003516.1,25731851


In [8]:
df[df["accession"].str.contains("NP_003509")]

,accession,variant,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,sequence,variant_under_consideration,sequence_accession,publication_id
276,NP_003509.1,cH2B.3_(Homo_sapiens),None,8346,H2BC10,9606.0,Homo sapiens,Chordata,Mammalia,Mammalia,None,MPEPAKSAPAPKKGSKKAVTKAQKKDGKKRKRSRKESYSVYVYKVL...,None,NP_003509.1,12408966
277,NP_003509.1,cH2B.3_(Homo_sapiens),None,8346,H2BC10,9606.0,Homo sapiens,Chordata,Mammalia,Mammalia,None,MPEPAKSAPAPKKGSKKAVTKAQKKDGKKRKRSRKESYSVYVYKVL...,None,NP_003509.1,25731851


In [10]:
query = "UPDATE sequence SET ncbi_gene_id=8346, hgnc_gene_name='H2BC10' WHERE accession = 'NP_003516.1'" 
print(query) 
cursor.execute(query) 

UPDATE sequence SET ncbi_gene_id=8346, hgnc_gene_name='H2BC10' WHERE accession = 'NP_003516.1'


In [11]:
query = (
    "SELECT * FROM sequence s LEFT JOIN sequence_has_publication sp "
    "ON s.accession = sp.sequence_accession "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["accession"].str.contains("NP_003516")]

,accession,variant,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,sequence,variant_under_consideration,sequence_accession,publication_id
290,NP_003516.1,cH2B.3_(Homo_sapiens),None,8346,H2BC10,9606.0,Homo sapiens,Chordata,Mammalia,Mammalia,None,MPEPAKSAPAPKKGSKKAVTKAQKKDGKKRKRSRKESYSVYVYKVL...,None,NP_003516.1,12408966
291,NP_003516.1,cH2B.3_(Homo_sapiens),None,8346,H2BC10,9606.0,Homo sapiens,Chordata,Mammalia,Mammalia,None,MPEPAKSAPAPKKGSKKAVTKAQKKDGKKRKRSRKESYSVYVYKVL...,None,NP_003516.1,25731851


In [12]:
query = "UPDATE sequence SET ncbi_gene_id=8339, hgnc_gene_name='H2BC8' WHERE accession = 'NP_003509.1'" 
print(query) 
cursor.execute(query) 

UPDATE sequence SET ncbi_gene_id=8339, hgnc_gene_name='H2BC8' WHERE accession = 'NP_003509.1'


In [13]:
query = (
    "SELECT * FROM sequence s LEFT JOIN sequence_has_publication sp "
    "ON s.accession = sp.sequence_accession "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["accession"].str.contains("NP_003509")]

,accession,variant,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,sequence,variant_under_consideration,sequence_accession,publication_id
276,NP_003509.1,cH2B.3_(Homo_sapiens),None,8339,H2BC8,9606.0,Homo sapiens,Chordata,Mammalia,Mammalia,None,MPEPAKSAPAPKKGSKKAVTKAQKKDGKKRKRSRKESYSVYVYKVL...,None,NP_003509.1,12408966
277,NP_003509.1,cH2B.3_(Homo_sapiens),None,8339,H2BC8,9606.0,Homo sapiens,Chordata,Mammalia,Mammalia,None,MPEPAKSAPAPKKGSKKAVTKAQKKDGKKRKRSRKESYSVYVYKVL...,None,NP_003509.1,25731851


In [15]:
df[df["accession"].str.contains("NP_003509")]["sequence"].values[0] == df[
    df["accession"].str.contains("NP_003516")
]["sequence"].values[0]

True

In [16]:
# Make sure data is committed to the database
conn.commit()

# Close connections

In [17]:
cursor.close()
conn.close()
tunnel.stop()